# Connecting to postgres

In [3]:
# we use the psycopg2 library as the standard connector lib.
from psycopg2 import connect

In [4]:
# setting the url of the database allows us to easily
# switch to other database configuraitons and locations
# we use the urllib parser to deconstruct the url. 
postgres_url = "postgres://127.0.0.1:5432/postgres"
from urllib.parse import urlparse
url = urlparse(postgres_url)

In [5]:
# return a connection we can use
def get_db_connection():
    return connect(
        host=url.hostname,
        port=url.port,
        user=url.username,
        password=url.password,
        database=url.path[1:]
    )

In [6]:
# create the connection and cursor
conn = get_db_connection()
cur = conn.cursor()

In [7]:
with get_db_connection() as conn:
    with conn.cursor() as cur:
        cur.execute("""
            create table cpus (
                id serial primary key,
                name varchar(255),
                cores int,
                threads int,
                price decimal                    
            );
        """)

In [8]:
with get_db_connection() as conn:
    with conn.cursor() as cur:
        cur.execute("""
            create table motherboards (
                    id serial primary key,
                    name varchar(255),
                    form_factor varchar(255),
                    price decimal
            );
        """)

# Parameterization

In [9]:
some_cpus = [
    ("bigballs", 12, 24, 3299.00),
    ("whiffmaerse", 10, 16, 2379.00),
    ("deeznuts", 18, 48, 7499.00),
    ("cheenies", 8, 12, 999.00),
]

some_motherboards = [
    {
        "name": "asus rog",
        "form_factor": "ATX",
        "price": 499.99
    },
    {
        "name": "gigabyte gigabollox",
        "form_factor": "EATX",
        "price": 379.99
    },
    {
        "name": "MSI edge ya bass",
        "form_factor": "ATX",
        "price": 849.99
    },
]

In [11]:
with get_db_connection() as conn:
    with conn.cursor() as cur:
        cur.executemany(""" 
            insert into cpus (name, cores, threads, price)
            values (%s, %s, %s, %s)
        """, some_cpus)

In [12]:
with get_db_connection() as conn:
    with conn.cursor() as cur:
        cur.executemany(""" 
            insert into motherboards (name, form_factor, price)
            values (%(name)s, %(form_factor)s, %(price)s);
        """, some_motherboards)

# Dynamic Queries

In [13]:
from psycopg2 import sql

In [14]:
table = "motherboards"
price = 2599.99
max_results = 3

In [15]:
with get_db_connection() as conn:
    with conn.cursor() as cur:
        cur.execute(sql.SQL("""
            select * 
            from {}
            where price <= {}
            limit {}                 
        """).format(sql.Identifier(table),
                    sql.Literal(price),
                    sql.Literal(max_results)
        ))
        print(cur.fetchall())

[(1, 'asus rog', 'ATX', Decimal('499.99')), (2, 'gigabyte gigabollox', 'EATX', Decimal('379.99')), (3, 'MSI edge ya bass', 'ATX', Decimal('849.99'))]


In [16]:
def get_items_by_price_threshold(item_type, price_threshold, max_results):
    exec_sql = sql.SQL(""" 
        select *
        from {}
        where price <= {}
        order by price desc
        limit {};                
    """).format(
        sql.Identifier(item_type),
        sql.Literal(price_threshold),
        sql.Literal(max_results)
    )

    with get_db_connection() as conn:
        with conn.cursor() as cur:
            cur.execute(exec_sql)
            return cur.fetchall()

In [17]:
get_items_by_price_threshold("cpus", 2500, 3)

[(2, 'whiffmaerse', 10, 16, Decimal('2379.0')),
 (4, 'cheenies', 8, 12, Decimal('999.0'))]

In [18]:
get_items_by_price_threshold("motherboards", 999, 3)

[(3, 'MSI edge ya bass', 'ATX', Decimal('849.99')),
 (6, 'MSI edge ya bass', 'ATX', Decimal('849.99')),
 (1, 'asus rog', 'ATX', Decimal('499.99'))]

In [19]:
# look at further reusability we want to have an extension to 
# the query function that facilitates the following:
# get_items("tablname", where={"field": value, "field": value})
# get_items("tablname", where={"field": value})
# get_items("tablname")

In [20]:
def get_items(item_type, where: dict = None, max_results: int = 3):
    where_caluse = sql.SQL("")

    if where:
        where_caluse = sql.SQL("where {}").format(
            sql.SQL(" and ").join([
                sql.SQL("{} = {}").format(
                    sql.Identifier(key), 
                    sql.Placeholder()
                ) for key in where.keys()
            ])
        )

    exec_sql = sql.SQL(""" 
        select *
        from {}
        {}
        order by price desc
        limit {};                
    """).format(
        sql.Identifier(item_type),
        where_caluse,
        sql.Literal(max_results)
    )

    with get_db_connection() as conn:
        with conn.cursor() as cur:
            cur.execute(exec_sql, list(where.values()))
            return cur.fetchall()

In [21]:
get_items("cpus", where={"cores": 10, "threads": 16})

[(2, 'whiffmaerse', 10, 16, Decimal('2379.0'))]

In [22]:
get_items("cpus", where={"cores": 18})

[(3, 'deeznuts', 18, 48, Decimal('7499.0'))]

# Dynamic Query refactor

In [23]:
def get_items(item_type, where: dict = None, max_results: int = 3):
    where_caluse = sql.SQL("")

    if where:
        # where_caluse = sql.SQL("where {}").format(
        #     sql.SQL(" and ").join([
        #         sql.SQL("{} = {}").format(
        #             sql.Identifier(key), 
        #             sql.Placeholder()
        #         ) for key in where.keys()
        #     ])
        # )

        conditions = [sql.SQL("{} = {}").format(
            sql.Identifier(key), 
            sql.Placeholder()
        ) for key in where.keys()]

        joined = sql.SQL(" and ").join(conditions)
        where_caluse = sql.SQL("where {}").format(joined)

    else: 
        where = {}
        where_caluse = sql.SQL("")

    exec_sql = sql.SQL(""" 
        select *
        from {}
        {}
        order by price desc
        limit {};                
    """).format(
        sql.Identifier(item_type),
        where_caluse,
        sql.Literal(max_results)
    )

    with get_db_connection() as conn:
        with conn.cursor() as cur:
            cur.execute(exec_sql, list(where.values()))
            return cur.fetchall()

In [24]:
get_items("cpus", where={"cores": 18})

[(3, 'deeznuts', 18, 48, Decimal('7499.0'))]

In [25]:
get_items("cpus", where={"threads": 24})

[(1, 'bigballs', 12, 24, Decimal('3299.0'))]